<a href="https://colab.research.google.com/github/manvento/Debriefing-automatico-grazie-alla-Computer-Vision-/blob/main/Text_recognizer_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text recognizer fine tuning

Questo notebook rappresenta gli experiments realizzati inizialmente per cercare una soluzione al riconoscimento dei display digitali.
Non è il modello definitivo, ma viene riportato per dare un esempio delle attività svolte.

## Fase iniziale per Google Colab

Questa sezione è necessaria soltanto se si usa Google Colab.

In [ ]:
# Here you must insert your Google Drive id to the data file, that is a compressed file containing the following entries.
# - pretrained_model (if you have any)
# - test
# - train 
# - valid
# last three folders must contain the annotated dataset for test, training and validation

data_id = '1KVJE_fv601htgE5l_2oCTjHaieMaCihm'

In [ ]:
if 'google.colab' in str(get_ipython()):
  print("You are using Google CoLab, so we need to donwload data compressed folder from Google Drive")
  !pip install -U -q PyDrive
  import os
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials

  # 1. Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # choose a local (colab) directory to store the data.
  local_download_path = os.path.expanduser('/content')
  try:
    os.makedirs(local_download_path)
  except: pass

  # download compressed file content
  f = drive.CreateFile({'id': data_id})
  fname = os.path.join(local_download_path, f['title'])
  f.GetContentFile(fname)

  import zipfile
  with zipfile.ZipFile(f['title'], 'r') as zip_ref:
      zip_ref.extractall('.')


## Installazione librerie aggiuntive

In [ ]:
if not 'google.colab' in str(get_ipython()):
  !pip install tensorflow~=2.6.0
  !pip install --upgrade matplotlib

!pip install keras_ocr
!pip install Pillow
!pip install opencv-python-headless==4.5.1.48

## Importing the needed libraries

In [ ]:
import os
import cv2
import glob
import math
import string
import shutil
import keras_ocr
import numpy as np
from PIL import Image

import tensorflow as tf
import matplotlib.pyplot as plt

# Caratteristiche dei display digitali:

- font specifici per il digitale
- basso contrasto (grigio su grigio o nero su verde scuro)
- presenza di rumore nell'immagine
- bassa risoluzione

In [ ]:
display(Image.open('test/images/01.jpg'))

# Definizione di funzioni di utilità


In [ ]:
# preprocessing and postprocessing

def fit(image):
    fitted = None
    width = 200
    height = 31
    x_scale = width / image.shape[1]
    y_scale = height / image.shape[0]
    if x_scale == 1 and y_scale == 1:
        fitted = image
        scale = 1
    elif x_scale <= y_scale:
        scale = width / image.shape[1]
        resize_width = width
        resize_height = (width / image.shape[1]) * image.shape[0]
    else:
        scale = height / image.shape[0]
        resize_height = height
        resize_width = scale * image.shape[1]

    if fitted is None:
        resize_width, resize_height = map(int, [resize_width, resize_height])
        fitted = np.zeros((height, width, 3), dtype="uint8")

        # opencv resize function raise an error if applied to a (1, 1, 3) image
        if image.shape[0] != 1 or image.shape[1] != 1:
            image = cv2.resize(image, dsize=(resize_width, resize_height))
            fitted[: image.shape[0], : image.shape[1]] = image[:height, :width]
    return fitted

def preprocess(img):
    img = fit(img)
    img = cv2.cvtColor(img, code=cv2.COLOR_RGB2GRAY)[..., np.newaxis] if img.shape[-1] == 3 else img
    img = img / 255
    img = img[np.newaxis, ...]
    img = np.asarray(img, dtype=np.float32)
    return img

def output_to_sentence(output, alphabet):
    return "".join([alphabet[i] if i != -1 else "" for i in output[0]])

In [ ]:
# helper function used to show the result on test images

def test_on_images(images, preproc_images, recognizer):
    num_cols = 4
    num_rows = math.ceil(len(images) / num_cols)

    x_dim = 25
    y_dim = math.ceil(num_rows / num_cols * 10)

    fig, axs = plt.subplots(num_rows, num_cols)
    fig.set_size_inches(x_dim, y_dim)


    for i in range(num_cols * num_rows):
        x = i // num_cols
        y = i % num_cols
        axs[x, y].set_xticks([])
        axs[x, y].set_yticks([])

        if i >= len(images):
            continue

        output = recognizer.prediction_model.predict(preproc_images[i])
        text_read = output_to_sentence(output, recognizer.alphabet)

        img = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
        axs[x, y].imshow(img)
        axs[x, y].set_title(f'{paths[i]} - {text_read}')

## Come si comporta il modello keras generico?

La prima fase è sempre quella di chiedersi: posso usare qualcosa di già pronto? Bene, allora vediamo come funziona il modello keras senza fitting.

In [ ]:
recognizer = keras_ocr.recognition.Recognizer()

In [ ]:
paths = sorted(glob.glob(os.path.join('test', 'images', '*'))[:20])
images = [cv2.imread(path) for path in paths]
preproc_images = [preprocess(img) for img in images]

In [ ]:
test_on_images(images, preproc_images, recognizer)

Come notiamo, il modello keras-ocr standard ottiene pessimi risultati nella lettura di font digitali, mentre funziona molto meglio in testi normali (il caso in alto a destra).

Qua non l'abbiamo mostrato, ma un altro problema con un modello senza fitting è dovuto al contrasto: tende a preferire i testi con contrasto alto dando meno importanza, ad esempio, ai display digitali. E' per questo che abbiamo realizzato un primo modello che esclude tutto ciò che sta fuori dal display stesso.

# Fase di addestramento

## parametri dell'algoritmo

In [ ]:
batch_size = 64
patience = 10
training_epochs = 1000

model_file_name = 'trained_model'

print(f'Parameters:\n' \
      f'\nbatch_size: {batch_size}' \
      f'\ntraining_epochs: {training_epochs}' \
      f'\npatience: {patience}')

## Definizione dell'alfabeto

Usiamo un generatore sintetico di immagini e gli indichiamo quali sono i testi ammissibili per il nostro caso

In [ ]:
# the alphabet is expanded with characters that are present in the displays and in the generated/tagged images

SPECIAL_CHARS = [' ', '+', '-', '.', ':', '=']

alphabet = string.digits + string.ascii_letters + ''.join(SPECIAL_CHARS)
custom_recognizer_alphabet = ''.join(sorted(set(alphabet.lower())))
print(f"The alphabet to recognize is: {custom_recognizer_alphabet}")

In [ ]:
print("Getting and compiling the recognizer...")

custom_recognizer = keras_ocr.recognition.Recognizer(
    alphabet=custom_recognizer_alphabet
)
custom_recognizer.compile()

_Come indicato dall'output, visto che l'alfabeto è diverso da quello del training originale, mantiene i pesi soltanto per i primi layer, quelli di feature extraction._

## Creiamo due liste di tuple contententi le immagini per training e validation

In [ ]:
img_labels = {}

for mode in ['train', 'valid']:
    with open(os.path.join(mode, 'pairs.txt')) as f:
        lines = f.read().splitlines()

    img_labels[mode] = [tuple(line.split('\t')) for line in lines]
    img_labels[mode] = [(img, None, label) for img, label in img_labels[mode]]

#### Vediamo, per esempio, come sono fatte queste tuple

In [ ]:
[print(line) for line in lines[:3]]

In [ ]:
display(Image.open(lines[0].split('\t')[0]))

### Ora creiamo degli image generator (che sono gli oggetti usati da Keras-OCR per leggere i dataset) in cui passiamo un le immagini che abbiamo generato oltre a quelle originali

In [ ]:
print('Building training and validation "image generators" from the datasets...')
(training_image_gen, training_steps), (validation_image_gen, validation_steps) = [
    (
        keras_ocr.datasets.get_recognizer_image_generator(
            labels=labels,
            height=custom_recognizer.model.input_shape[1],
            width=custom_recognizer.model.input_shape[2],
            alphabet=custom_recognizer_alphabet
        ),
        len(labels) // batch_size
    ) for labels in [img_labels['train'], img_labels['valid']]
]

training_gen, validation_gen = [
    custom_recognizer.get_batch_generator(
        image_generator=image_generator,
        batch_size=batch_size
    )
    for image_generator in [training_image_gen, validation_image_gen]
]

#### Ora definiamo delle callback utili per generare log da visualizzare in fase di training e il check sulla patience, per l'early stopping, ovvero terminiamo il training quando non ci sono miglioramenti nella validation loss per _n_ cicli, dove _n_ è proprio la patience.

In [ ]:
if os.path.isdir(model_file_name):
    shutil.rmtree(model_file_name)

os.mkdir(model_file_name)
chk_file = os.path.join('.', model_file_name)
log_file = os.path.join('.', model_file_name, 'epochs.csv')

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, restore_best_weights=False),
    tf.keras.callbacks.ModelCheckpoint(chk_file, monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.CSVLogger(log_file)
]


#### A questo punto si può procedere con il training vero e proprio.

In [ ]:
print('Training the custom recognizer...')

# custom_recognizer.training_model.fit(
#     training_gen,
#     steps_per_epoch=training_steps,
#     validation_steps=validation_steps,
#     validation_data=validation_gen,
#     callbacks=callbacks,
#     epochs=training_epochs
# )

# Model testing

Ora carichiamo i pesi del modello appena addestrato con l'alfabeto definito, e verifichiamo se siamo riusciti ad ottenere miglioramenti rispetto a Keras-OCR senza fine tuning.

In [ ]:
pretrained_recognizer = keras_ocr.recognition.Recognizer(
        alphabet=custom_recognizer_alphabet
    )
pretrained_recognizer.compile()
pretrained_recognizer.model.load_weights('pretrained_model')

In [ ]:
test_on_images(images, preproc_images, pretrained_recognizer)